# TS Prophet
## Data split in training and test. The last 365 days are the test data.

In [ ]:
from audioop import cross
import itertools
from matplotlib import units
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import add_changepoints_to_plot
from prophet.plot import plot_cross_validation_metric
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.serialize import model_to_json, model_from_json

plt.style.use('fivethirtyeight')

from sklearn.metrics import mean_absolute_error,mean_squared_error

from green_city.utils import metrics_dict, datetime2index, index2datetime

# Libraries to reduce Prophet verbose output
import warnings
import logging
import os

%matplotlib inline

In [ ]:
## MLFLOW ##
import mlflow
from green_city.mlflow_config import get_mlflow_config

flow_conf = get_mlflow_config()
tracking_uri = flow_conf["TRACKING_URI"]
mlflow.set_tracking_uri(flow_conf["TRACKING_URI"])
mlflow.set_experiment(flow_conf["EXPERIMENT_NAME"]);

In [ ]:
def is_winter_season(ds):
    """Classifies the dates into Winter or Summer

    Args:
        ds (datetime series): A Pandas datetime series
    """
    date = pd.to_datetime(ds)
    return(date.month < 4 or date.month > 9)


In [ ]:
## DB CONNECTION ##
from sqlalchemy import create_engine
from decouple import Config, RepositoryEnv

config = Config(RepositoryEnv("../.db_credentials"))

db_connection_credentials = {
    "database": config('POSTGRES_DB'),
    "user": config('POSTGRES_USER'),
    "password": config('POSTGRES_PASSWORD'),
    "host": config('POSTGRES_HOST'),
    "port": config('POSTGRES_PORT'),
}
DB_STRING = "postgresql://{user}:{password}@{host}:{port}/{database}".format(**db_connection_credentials)
db = create_engine(DB_STRING)

### Load data and feature engineering

In [ ]:
# building = 5
# # Load data
# df_building = pd.read_csv(f"../data/preprocessed/Building_{building}.csv").astype({'datetime': 'datetime64'})#.set_index('datetime')

In [ ]:
building = "All"
# Load data
df_building = pd.read_csv(f"../data/preprocessed/Agg_buildings.csv").astype({'datetime': 'datetime64'})#.set_index('datetime')

In [ ]:
df_building = df_building.rename(columns={'datetime':'ds',
                                    'net_load_kW':'y'})
# df.head(5)

In [ ]:
df_building.y.describe()

In [ ]:
df_building['winter'] = df_building['ds'].apply(is_winter_season)
df_building['summer'] = ~df_building['ds'].apply(is_winter_season)

In [ ]:
df_holiday = df_building[['ds','holiday']].query('holiday')
df_holiday['holiday'] = 'Holiday'
# df_holiday

## In the development stage, it was explored to add the workday as part of the  ##
## seasonality. It did not had an effect. Including the holiday did.            ##
# df_workday = df[['ds','workday']].query('workday')
# df_workday['holiday'] = 'Workday'
# df_workday = df_workday.drop('workday', axis=1)

# df_holidays = pd.concat([df_holiday, df_workday], ignore_index=True)

In [ ]:
pred_indices = [32135, 33311, 26478, 33357, 30387, 30794, 31800, 28783]
# pred_indices = [32135]

## Prophet simple model. Three years training

### Split data in training and test data

In [ ]:
df_simple_model_train = df_building[:365*3*24]
df_simple_model_test = df_building[365*3*24:]

### Simple Prophed model. Specify 95% of uncertainty and additive model. (Default 80% and additive model)

In [ ]:
my_model_simple = Prophet(interval_width = 0.95, seasonality_mode="additive")

In [ ]:
my_model_simple.fit(df_simple_model_train)

In [ ]:
# Save model
with open('../models/my_model_simple.json', 'w') as fout:
    fout.write(model_to_json(my_model_simple))  

### Predicting 8 days with basic model

In [ ]:
def my_pred(df,model_name, params, pred_index):
    """Forecast a specified interval into a Prophet model. 

    Args:
        df (dataframe)    : A pandas dataframe with the form required by Prophet. Must include the training and testing data.
        model_name (str)  : A string with a unique description of the model to evaluate.
        params(list) : A list of dictionaries with the parameters definition. 
        pred_index (list) : A list with the indexes of the intervals to predict. ("Test data"). These indexes must exist in the df variable.

    Returns:
        forecasts (dataframe)           : A dataframe with the forecasted values from the interval of interest. 
        ls_best_params (list)           : A list with the best hyperpameters and its evaluation metrics; mse and rsme.
        my_model(fitted Prophet model)  : A fitted prophet model. The model utilized to forecast the data. 
        ls_publish                      : A list with the information to put in mlflow and the SQL server. Includes the forecasts dataframe,
                                          the msa, mse and r2_score.
    """

    # ls_best_params = []
    ls_publish = []
    for index in pred_index:
        df_train=df.loc[:index] # dataframe to train the model and its optimization (this was a mistake)
        df_future = df.loc[index+1:index+24]
        with mlflow.start_run(run_name=model_name) as r:
            ### Predict ##
            # best_params, tuning_results = my_prophet_mse(df_train, all_params=parameters) # The optimized parameters are coming form the 3 years training data
            # ls_best_params.append([{'index':index}, best_params, tuning_results]) # This is not required anymore.
            my_model = Prophet(**params)
            my_model.fit(df_train)

            ## Save Model##
            with open(f'../models/{model_name}_{index}.json', 'w') as fout:
                fout.write(model_to_json(my_model_simple))  

            df_forecast = my_model.predict(df_future)
            df_forecast.index = df_future.index
            df_forecast.index.name = 'id'
            df_forecast['error'] = df_future.y - df_forecast.yhat
            df_forecast = df_forecast[['yhat', 'error']]
            
            ## Evaluate ##
            metrics = metrics_dict(df_future.y, df_forecast.yhat, ["mae", "mse", "r2_score"])
            print(index)
            print("mae: {mae}, mse: {mse}, r2: {r2_score}".format(**metrics))
            # print(ls_best_params)
            mlflow.log_metrics(metrics)

            ## Parameters log ##
            mlflow_params = {
                'building_nr': building,
                'datetime':index2datetime(index)
            }
            mlflow.log_params(mlflow_params)

            forecasts = df_forecast[['yhat']].assign(run_id = r.info.run_id).rename(columns={"yhat": "prediction"})
            # forecasts.to_sql("forecast", con=db, if_exists="append")
            ls_publish.append([{'index':index,'metrics':metrics, 'params':params, 'forecast':forecasts}])
    return forecasts, my_model, ls_publish

In [ ]:
# Use MSE for tunning 
param_grid_simple = {
    'interval_width': [0.95],
    'seasonality_mode':['additive'],
    'changepoint_prior_scale':[0.05],
    'seasonality_prior_scale':[10.0],
    'holidays_prior_scale':[0.01],
    'changepoint_range':[0.8]
}
my_simple_params = [dict(zip(param_grid_simple.keys(), v)) for v in itertools.product(*param_grid_simple.values())]


In [ ]:
pred_mse_simpleMod, \
    my_model_simpleMod, \
    ls_publish_mse_simpleMod = my_pred(df = df_building,
        model_name='Prophet Simple Model', 
        params=my_simple_params[0],
        pred_index=pred_indices)

## Optimize modeling. Find best hyper parameters

In [ ]:
def my_prophet_find_hyperparameter(df, all_params):
    """Determine the best hyper parameters for a Prophet model.
    It uses a base Prophet model. 

    Args:
        df (dataframe): A pandas dataframe with the required Prophet structure
        all_params (list of dictionaries): A list of dictionaries with the parameters definition. 

    Returns:
        best_params: a list with the best parameters
        tuning_results: A dataframe with the mse and rmse evaluation of the parameters combinations
    """
    warnings.simplefilter("ignore", DeprecationWarning)
    warnings.simplefilter("ignore", FutureWarning)
    logging.getLogger('prophet').setLevel(logging.ERROR) #Notice that i had modified the name from 'fbprophet' to just 'prophet'
    mses = []
    rmses = []
    c=0
    for params in all_params:
        c =+ c
        m = Prophet(**params)
        m.fit(df)
        ## Save Model##
        with open(f'../models/Prophet_paramSearch_{c}.json', 'w') as fout:
            fout.write(model_to_json(my_model_simple))  
        df_cv = cross_validation(m, horizon='180 days', parallel='processes')
        df_p = performance_metrics(df_cv, rolling_window=1)
        mses.append(df_p['mse'].values[0])
        rmses.append(df_p['rmse'].values[0])
    tuning_results = pd.DataFrame(all_params)
    tuning_results['mse'] = mses
    tuning_results['rmse'] = rmses
    print(tuning_results)
    best_params=all_params[np.argmin(mses)]
    print(best_params)
    return best_params, tuning_results


In [ ]:
# Parameter optimization. Tunning CV with MSE 
param_grid = {
    'interval_width': [0.95],
    'seasonality_mode':['additive'],
    'changepoint_prior_scale':[0.0001, 0.01, 0.05, 0.1, 0.5], #Parameters like this (regularization penalties; this is effectively a lasso penalty) are often tuned on a log scale.
    'seasonality_prior_scale':[0.01, 0.1, 1.0, 10.0], #This likely also makes sense on a log scale, since it is effectively an L2 penalty like in ridge regression.
    'holidays_prior_scale':[0.01, 0.1, 1.0, 10.0],
    'changepoint_range':[0.8,0.9]
}
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

## Do not run. It takes 95m. The results are in the next code block

In [ ]:
my_optimized_parameters, \
    my_optimized_param_tunning_results = \
        my_prophet_find_hyperparameter(df_simple_model_train, all_params)

In [ ]:
my_optimized_parameters

In [ ]:
# my_optimized_parameters = {'interval_width': 0.95,
#     'seasonality_mode': 'additive',
#     'changepoint_prior_scale': 0.0001,
#     'seasonality_prior_scale': 0.01,
#     'holidays_prior_scale': 0.01,
#     'changepoint_range': 0.9}

In [ ]:
# # Save the hyperparameters results from cross validation. RMSE and MSE are documented.
# my_optimized_param_tunning_results
# my_optimized_param_tunning_results.to_csv('../data/preprocessed/prophet_best_hyperparameters_bld_5.csv',
#     sep=";", index_label='Index')

In [ ]:
# Save the hyperparameters results from cross validation. RMSE and MSE are documented.
my_optimized_param_tunning_results
my_optimized_param_tunning_results.to_csv('../data/preprocessed/prophet_best_hyperparameters_allbld.csv',
    sep=";", index_label='Index')

### Parameters that can be tuned

**changepoint_prior_scale**: This is probably the most impactful parameter. It determines the flexibility of the trend, and in particular how much the trend changes at the trend changepoints. As described in this documentation, if it is too small, the trend will be underfit and variance that should have been modeled with trend changes will instead end up being handled with the noise term. If it is too large, the trend will overfit and in the most extreme case you can end up with the trend capturing yearly seasonality. The default of 0.05 works for many time series, but this could be tuned; a range of [0.001, 0.5] would likely be about right. Parameters like this (regularization penalties; this is effectively a lasso penalty) are often tuned on a log scale.

**seasonality_prior_scale**: This parameter controls the flexibility of the seasonality. Similarly, a large value allows the seasonality to fit large fluctuations, a small value shrinks the magnitude of the seasonality. The default is 10., which applies basically no regularization. That is because we very rarely see overfitting here (there’s inherent regularization with the fact that it is being modeled with a truncated Fourier series, so it’s essentially low-pass filtered). A reasonable range for tuning it would probably be [0.01, 10]; when set to 0.01 you should find that the magnitude of seasonality is forced to be very small. This likely also makes sense on a log scale, since it is effectively an L2 penalty like in ridge regression.

**holidays_prior_scale**: This controls flexibility to fit holiday effects. Similar to seasonality_prior_scale, it defaults to 10.0 which applies basically no regularization, since we usually have multiple observations of holidays and can do a good job of estimating their effects. This could also be tuned on a range of [0.01, 10] as with seasonality_prior_scale.

**seasonality_mode**: Options are ['additive', 'multiplicative']. Default is 'additive', but many business time series will have multiplicative seasonality. This is best identified just from looking at the time series and seeing if the magnitude of seasonal fluctuations grows with the magnitude of the time series (see the documentation here on multiplicative seasonality), but when that isn’t possible, it could be tuned.

In [ ]:
pred_simpleMod_Opt, \
        my_model_simpleMod_Opt, \
        ls_publish_simpleMod_Opt = my_pred(df = df_building,
        model_name='Prophet Simple Model Optimized Parameters - Tunning with MSE', 
        params=my_optimized_parameters,
        pred_index=pred_indices)

### Add holidays

In [ ]:
param_grid = {
    'interval_width': [0.95],
    'seasonality_mode':['additive'],
    'changepoint_prior_scale':[0.0001],
    'seasonality_prior_scale':[0.01],
    'holidays_prior_scale':[0.01],
    'changepoint_range':[0.9],
    'holidays':[df_holiday]
}
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

pred_Opt_holidays, \
    m_best_Opt_holidays, \
    ls_publish_Opt_holidays = my_pred(df = df_building,
        model_name='Prophet Holidays-Optimized-Parameters - Tunning with MSE', 
        params=my_optimized_parameters,
        pred_index=pred_indices)

### Add outdoor temperature, humidity, diffuse and direct solar energy as additional regressor. 
### The temperature, humidity and solar energy data is in the original dataset.

In [ ]:
def my_pred_reg(df,model_name, params, pred_index):
    """Forecast a specified interval into a Prophet model. 

    Args:
        df (dataframe)    : A pandas dataframe with the form required by Prophet. Must include the training and testing data.
        model_name (str)  : A string with a unique description of the model to evaluate.
        params(list) : A list of dictionaries with the parameters definition. 
        pred_index (list) : A list with the indexes of the intervals to predict. ("Test data"). These indexes must exist in the df variable.

    Returns:
        forecasts (dataframe)           : A dataframe with the forecasted values from the interval of interest. 
        ls_best_params (list)           : A list with the best hyperpameters and its evaluation metrics; mse and rsme.
        my_model(fitted Prophet model)  : A fitted prophet model. The model utilized to forecast the data. 
        ls_publish                      : A list with the information to put in mlflow and the SQL server. Includes the forecasts dataframe,
                                          the msa, mse and r2_score.
    """

    # ls_best_params = []
    ls_publish = []
    for index in pred_index:
        df_train=df.loc[:index] # dataframe to train the model and its optimization (this was a mistake)
        df_future = df.loc[index+1:index+24]
        with mlflow.start_run(run_name=model_name) as r:
            ### Predict ##
            # best_params, tuning_results = my_prophet_mse(df_train, all_params=parameters) # The optimized parameters are coming form the 3 years training data
            # ls_best_params.append([{'index':index}, best_params, tuning_results]) # This is not required anymore.
            my_model = Prophet(**params)
            my_model.add_regressor(name='outdoor_temp')
            my_model.add_regressor(name='outdoor_hum')
            my_model.add_regressor(name='diffuse_solar_W_m2')
            my_model.add_regressor(name='direct_solar_W_m2')
            my_model.add_regressor(name='pred_24h_outdoor_temp')
            my_model.add_regressor(name='pred_24h_outdoor_hum')
            my_model.add_regressor(name='pred_24h_diffuse_solar_W_m2')
            my_model.add_regressor(name='pred_24h_direct_solar_W_m2')
            my_model.fit(df_train) 

            ## Save Model##
            with open(f'../models/{model_name}_{index}.json', 'w') as fout:
                fout.write(model_to_json(my_model_simple))  
            
            df_forecast = my_model.predict(df_future)
            df_forecast.index = df_future.index
            df_forecast.index.name = 'id'
            df_forecast['error'] = df_future.y - df_forecast.yhat
            df_forecast = df_forecast[['yhat', 'error']]
                 
            ## Evaluate ##
            metrics = metrics_dict(df_future.y, df_forecast.yhat, ["mae", "mse", "r2_score"])
            print(index)
            print("mae: {mae}, mse: {mse}, r2: {r2_score}".format(**metrics))
            # print(ls_best_params)
            mlflow.log_metrics(metrics)

            ## Parameters log ##
            mlflow_params = {
                'building_nr': building,
                'datetime':index2datetime(index)
            }
            mlflow.log_params(mlflow_params)

            forecasts = df_forecast[['yhat']].assign(run_id = r.info.run_id).rename(columns={"yhat": "prediction"})
            # forecasts.to_sql("forecast", con=db, if_exists="append")
            ls_publish.append([{'index':index,'metrics':metrics, 'params':params, 'forecast':forecasts}])
    return forecasts, my_model, ls_publish

In [ ]:
param_grid = {
    'interval_width': [0.95],
    'seasonality_mode':['additive'],
    'changepoint_prior_scale':[0.0001],
    'seasonality_prior_scale':[0.01],
    'holidays_prior_scale':[0.01],
    'changepoint_range':[0.9],
    'holidays':[df_holiday]
}
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

pred_best_WkMod_RegMod_Opt, \
    m_best_RegMod_Opt, \
    ls_publish_best_RegMod_Opt = my_pred_reg(df=df_building,
        model_name='Prophet Holidays-Regressors-Optimized-Parameters - Tunning with MSE', 
        params=all_params[0],
        pred_index=pred_indices)

### Stop. The following results are dubious at the moment. (01.08.2022)

### Assume different weekly sesonalities. Summer vs Winter

In [ ]:
def my_pred_reg_wk(df,model_name, params, pred_index):
    """Forecast a specified interval into a Prophet model. 

    Args:
        df (dataframe)    : A pandas dataframe with the form required by Prophet. Must include the training and testing data.
        model_name (str)  : A string with a unique description of the model to evaluate.
        params(list) : A list of dictionaries with the parameters definition. 
        pred_index (list) : A list with the indexes of the intervals to predict. ("Test data"). These indexes must exist in the df variable.

    Returns:
        forecasts (dataframe)           : A dataframe with the forecasted values from the interval of interest. 
        ls_best_params (list)           : A list with the best hyperpameters and its evaluation metrics; mse and rsme.
        my_model(fitted Prophet model)  : A fitted prophet model. The model utilized to forecast the data. 
        ls_publish                      : A list with the information to put in mlflow and the SQL server. Includes the forecasts dataframe,
                                          the msa, mse and r2_score.
    """

    # ls_best_params = []
    ls_publish = []
    for index in pred_index:
        df_train=df.loc[:index] # dataframe to train the model and its optimization (this was a mistake)
        df_future = df.loc[index+1:index+24]
        with mlflow.start_run(run_name=model_name) as r:
            ### Predict ##
            # best_params, tuning_results = my_prophet_mse(df_train, all_params=parameters) # The optimized parameters are coming form the 3 years training data
            # ls_best_params.append([{'index':index}, best_params, tuning_results]) # This is not required anymore.
            my_model = Prophet(**params, weekly_seasonality=False)
            my_model.add_regressor(name='outdoor_temp')
            my_model.add_regressor(name='outdoor_hum')
            my_model.add_regressor(name='diffuse_solar_W_m2')
            my_model.add_regressor(name='direct_solar_W_m2')
            my_model.add_regressor(name='pred_24h_outdoor_temp')
            my_model.add_regressor(name='pred_24h_outdoor_hum')
            my_model.add_regressor(name='pred_24h_diffuse_solar_W_m2')
            my_model.add_regressor(name='pred_24h_direct_solar_W_m2')
            my_model.add_seasonality(name='weekly_on_winter', period = 7, fourier_order= 10, condition_name='winter')
            my_model.add_seasonality(name='weekly_on_summer', period = 7, fourier_order= 10, condition_name='summer')
            my_model.fit(df_train) 

            ## Save Model##
            with open(f'../models/{model_name}_{index}.json', 'w') as fout:
                fout.write(model_to_json(my_model_simple))  

            df_forecast = my_model.predict(df_future)
            df_forecast.index = df_future.index
            df_forecast.index.name = 'id'
            df_forecast['error'] = df_future.y - df_forecast.yhat
            df_forecast = df_forecast[['yhat', 'error']]
            
            ## Evaluate ##
            metrics = metrics_dict(df_future.y, df_forecast.yhat, ["mae", "mse", "r2_score"])
            print(index)
            print("mae: {mae}, mse: {mse}, r2: {r2_score}".format(**metrics))
            # print(ls_best_params)
            mlflow.log_metrics(metrics)

            ## Parameters log ##
            mlflow_params = {
                'building_nr': building,
                'datetime':index2datetime(index)
            }
            mlflow.log_params(mlflow_params)

            forecasts = df_forecast[['yhat']].assign(run_id = r.info.run_id).rename(columns={"yhat": "prediction"})
            # forecasts.to_sql("forecast", con=db, if_exists="append")
            ls_publish.append([{'index':index,'metrics':metrics, 'params':params, 'forecast':forecasts}])
    return forecasts, my_model, ls_publish

In [ ]:
param_grid = {
    'interval_width': [0.95],
    'seasonality_mode':['additive'],
    'changepoint_prior_scale':[0.0001],
    'seasonality_prior_scale':[0.01],
    'holidays_prior_scale':[0.01],
    'changepoint_range':[0.9],
    'holidays':[df_holiday]
}
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

pred_best_WkMod_RegMod_Opt, \
    m_best_WkMod_RegMod_Opt, \
    ls_publish_best_WkMod_RegMod_Opt = my_pred_reg_wk(df=df_building,
        model_name='Prophet Holidays-Wkl-Regressors-Optimized-Parameters - Tunning with MSE', 
        params=all_params[0],
        pred_index=pred_indices)